In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from sentence_transformers import util

/Users/valeriaalesnikova/Desktop/bootcamp/ghjtrn/smart_book_search/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка модели и токенизатора BERT
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [4]:
books = pd.read_csv('/Users/valeriaalesnikova/Desktop/bootcamp/ghjtrn/smart_book_search/data/data_withot_loveread.csv')
books

,Unnamed: 0,page_url,image_url,author,title,annotation,cleaned_text,lemmatized_text,cleaned_title,text_length
0,0,https://www.biblio-globus.ru//product/10957168,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,Ричард Третий и Генрих Восьмой глазами Шекспира,Расследование жизни и деяний двух самых опороч...,расследование жизни деяний самых опороченных в...,расследование жизнь деяние самый опорочить воз...,ричард третий и генрих восьмой глазами ше...,1046
1,1,https://www.biblio-globus.ru//product/10997075,https://static1.bgshop.ru/imagehandler.ashx?fi...,Михалкова Е.И.,Знак истинного пути,Сыщики Макар Илюшин и Сергей Бабкин расследуют...,сыщики макар илюшин сергей бабкин расследуют п...,сыщик макар илюшин сергей бабкин расследовать ...,знак истинного пути,242
2,2,https://www.biblio-globus.ru//product/10941892,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,Генрих Шестой глазами Шекспира,Генрих Шестой – самая трагическая фигура на ан...,генрих самая трагическая фигура английском пре...,генрих самый трагический фигура английский пре...,генрих шестой глазами шекспира,1005
3,3,https://www.biblio-globus.ru//product/10978116,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,"Иоанн Безземельный, Эдуард Третий и Ричард Вто...",Истории трех знаменитых королей под одной обло...,истории трех знаменитых королей обложкой посво...,история три знаменитый король обложка посвоему...,иоанн безземельный эдуард третий и ричар...,1050
4,4,https://www.biblio-globus.ru//product/10845007,https://static1.bgshop.ru/extraimagehandler.as...,Маринина А.,Шпаргалка для ленивых любителей истории. Корол...,Первая книга в необычном для Александры Марини...,первая необычном александры марининой жанре по...,первый необычный александра маринин жанр попул...,шпаргалка для ленивых любителей истории. кор...,598
...,...,...,...,...,...,...,...,...,...,...
3590,3935,Преступления продолжаются\,и количество трупов в виде мумий растет. Элиз...,которая повлекла за собой нечто более зловещее\,чем она могла себе представить. Ее репортажи ...,и каждая новая догадка обличает богатых и вли...,новая догадка обличает богатых влиятельных гра...,новый догадка обличать богатый влиятельный гра...,чем она могла себе представить. ее репортажи...,144
3591,3953,https://www.biblio-globus.ru//product/10958255,https://static1.bgshop.ru/imagehandler.ashx?fi...,Кинг С.,"Извлечение троих: из цикла \""Темная Башня\""",Цикл «Темная Башня» — это opus magnum — лучшая...,цикл темная башня лучшая стивена кинга история...,цикл тёмный башня хороший стивен кинг история ...,"извлечение троих: из цикла \"" темная башня\""",85
3592,3954,Юный Роланд — последний благородный рыцарь в м...,«сдвинувшемся с места». Ему во что бы то ни с...,краеугольный камень мироздания. Но прежде Рол...,как говорят карты Таро\,в современную Америку на поиски помощников. В...,современную америку поиски помощников смогут п...,современный америка поиск помощник смочь проти...,как говорят карты таро\,92
3593,3962,Антиквар изучает рукопись\,а в городе происходят загадочные и порой откр...,оживают обезглавленные мертвецы\,улицы наполняются толпами зомби. И похоже на то\,что главной целью нечисти становится именно К...,главной целью нечисти становится кирилл неболь...,главный цель нечисть становиться кирилл неболь...,улицы наполняются толпами зомби. и похоже на...,181


In [5]:
# Максимальная длина текста
MAX_LEN = 300

In [6]:
# Функция для встраивания текста с использованием BERT
def embed_bert_cls(text, model=model, tokenizer=tokenizer):
    t = tokenizer(text,
                  padding=True,
                  truncation=True,
                  return_tensors='pt',
                  max_length=MAX_LEN)
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().squeeze()

In [7]:
# Функция для обработки текста, чтобы все значения были строками
def preprocess_text(text):
    if isinstance(text, str):
        return text
    elif pd.isna(text):
        return ""
    else:
        return str(text)

# Применение функции к колонке 'annotation'
books['lemmatized_text'] = books['lemmatized_text'].apply(preprocess_text)

# Вычисление векторов для всех текстов в DataFrame
embeddings = []
for text in books['lemmatized_text']:  # Предполагаем, что тексты находятся в колонке 'annotation'
    embedding = embed_bert_cls(text)
    embeddings.append(embedding.numpy())

# Преобразование списка векторов в массив numpy
embeddings_array = np.stack(embeddings)

# Сохранение векторов в текстовый файл
np.savetxt('embeddings.txt', embeddings_array)

In [8]:
# Загрузка предварительно вычисленных векторов
embeddings = np.loadtxt('embeddings.txt')
embeddings_tensor = [torch.tensor(embedding) for embedding in embeddings]

# Создание индекса Faiss
embeddings_matrix = np.stack(embeddings)
index = faiss.IndexFlatIP(embeddings_matrix.shape[1])
index.add(embeddings_matrix)

# Текст запроса
text = 'Между сыщиком и преступником начинается жесткая игра'

# Встраивание запроса и поиск ближайших векторов с использованием Faiss
query_embedding = embed_bert_cls(text)
query_embedding = query_embedding.numpy().astype('float32')
k, indices = index.search(np.expand_dims(query_embedding, axis=0), 5)

# Вывод результатов
for i in indices[0]:
    print(books['title'][i])


Венчание со страхом
Фартовый город
Ловцы черных душ. Дело глазника
Убийство на улице Роз
Апология здравого смысла
